In [12]:
# These are the libraries will be used.
import torchvision.models as models
from PIL import Image
import pandas
from torchvision import transforms
import torch.nn as nn
import time
import torch 
import matplotlib.pylab as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
import h5py
import os
import glob
torch.manual_seed(0)

from tqdm import tqdm
from dataSet import *
from unet_model import *
from torch.utils.data import DataLoader, random_split


In [13]:
# Change these directories as needed

# Image Directories 
# image
dir_image='/home/matmatius/Desktop/otsuImages/originalrgb/'
# mask
dir_mask='/home/matmatius/Desktop/otsuImages/masksrgb/'
# scale image

In [14]:
dataset=Dataset(dir_image,dir_mask)

In [15]:
dataset[0]['image'].view(-1,1,128,128).shape

torch.Size([1, 1, 128, 128])

In [16]:
torch.max(dataset[0]['mask'])

tensor(1)

In [17]:
model=UNet(1,2)

In [18]:
model

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(128, eps=1e-05, moment

In [19]:
x=dataset[0]['image'].view(-1,1,128,128).float()
x.shape

torch.Size([1, 1, 128, 128])

In [20]:
z=model(x)
print(z.shape)
#z=torch.transpose(z,1,3)
#z=z[0]
#print(z.shape)

torch.Size([1, 2, 128, 128])


In [21]:
_, yhat = torch.max(z.data, 1)#computes soft max

In [22]:
yhat.shape

torch.Size([1, 128, 128])

In [23]:
yhat.view(1,-1).shape

torch.Size([1, 16384])

In [24]:
dataset[0]['mask'].shape

torch.Size([1, 128, 128])

In [25]:
#loss(z[0],dataset[0]['mask'].view(-1,1,128,128))

In [26]:
loss = nn.CrossEntropyLoss()

In [27]:
input = torch.randn(3, 2, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(2)
output = loss(input, target)

In [28]:
input.shape

torch.Size([3, 2])

In [29]:
input

tensor([[-0.6341, -0.8350],
        [-0.3637,  0.0403],
        [-0.9763, -0.1906]], requires_grad=True)

In [30]:
target

tensor([0, 1, 0])

In [31]:
output

tensor(0.7568, grad_fn=<NllLossBackward>)

In [32]:
x=np.array([2,1,1])

In [33]:
np.exp(x)

array([7.3890561 , 2.71828183, 2.71828183])

In [34]:
def softmax(x):
    return np.exp(x)/np.sum(np.exp(x),axis=0)

In [35]:
softmax(x)

array([0.57611688, 0.21194156, 0.21194156])

In [36]:
x=torch.tensor([2,1,1], dtype=float)
torch.softmax(x,dim=0)

tensor([0.5761, 0.2119, 0.2119], dtype=torch.float64)

In [37]:
x=torch.tensor([[[1,2,3],[-1,4,-5]],[[3,-2,1],[3,7,-3]]],dtype=float)
print(x)

tensor([[[ 1.,  2.,  3.],
         [-1.,  4., -5.]],

        [[ 3., -2.,  1.],
         [ 3.,  7., -3.]]], dtype=torch.float64)


In [38]:
y=torch.softmax(x,dim=1)
print(y, y.shape)

tensor([[[8.8080e-01, 1.1920e-01, 9.9966e-01],
         [1.1920e-01, 8.8080e-01, 3.3535e-04]],

        [[5.0000e-01, 1.2339e-04, 9.8201e-01],
         [5.0000e-01, 9.9988e-01, 1.7986e-02]]], dtype=torch.float64) torch.Size([2, 2, 3])


In [39]:
print(x.shape)
sum=torch.sum(x,0)
print(sum,sum.shape)

torch.Size([2, 2, 3])
tensor([[ 4.,  0.,  4.],
        [ 2., 11., -8.]], dtype=torch.float64) torch.Size([2, 3])


In [40]:
y=np.array([1,0,0])
ygood=np.array([0.7,0.2,0.1])
ybad=np.array([0.1,0.3,0.6])

In [41]:
np.log(ygood)

array([-0.35667494, -1.60943791, -2.30258509])

In [42]:
-np.sum(y*np.log(ygood))

0.35667494393873245

In [43]:
-np.sum(y*np.log(ybad))

2.3025850929940455

In [44]:
np.log(0.1)

-2.3025850929940455

In [45]:
def cross_entropy(actual,predicted):
    return -np.sum(actual*np.log(predicted))

In [46]:
cross_entropy(y,ygood)

0.35667494393873245

In [47]:
cross_entropy(y,ybad)

2.3025850929940455

In [48]:
loss=nn.CrossEntropyLoss()

In [49]:
#3 samples
Y=torch.tensor([1,0,1,1,0])

# number_of_samples x number_of_clasess =3 x 2
Ygood=torch.tensor([[1,2.1],[2,1],[0.1,3],[1,4],[2,3]])
Ybad=torch.tensor([[3,2.1],[2,11],[4.1,3],[1,4],[2,3]])

In [50]:
print(Y,Y.shape)
print(Ygood,Ygood.shape)
print(Ybad,Ybad.shape)

tensor([1, 0, 1, 1, 0]) torch.Size([5])
tensor([[1.0000, 2.1000],
        [2.0000, 1.0000],
        [0.1000, 3.0000],
        [1.0000, 4.0000],
        [2.0000, 3.0000]]) torch.Size([5, 2])
tensor([[ 3.0000,  2.1000],
        [ 2.0000, 11.0000],
        [ 4.1000,  3.0000],
        [ 1.0000,  4.0000],
        [ 2.0000,  3.0000]]) torch.Size([5, 2])


In [51]:
l1=loss(Ygood,Y)
l2=loss(Ybad,Y)
print(l1,l2)

tensor(0.4032) tensor(2.5981)


In [52]:
x=dataset[0]['image']
y=dataset[0]['mask']

In [53]:
z=model(x.view(-1,1,128,128).float())[0]

In [54]:
z[:,0:3,0:3]

tensor([[[0.0761, 0.0382, 0.2798],
         [0.3035, 0.0857, 0.2666],
         [0.1222, 0.1765, 0.1431]],

        [[0.2141, 0.0117, 0.0949],
         [0.2689, 0.1247, 0.2399],
         [0.3284, 0.3797, 0.1583]]], grad_fn=<SliceBackward>)

In [55]:
z[:,0:3,0:3].shape

torch.Size([2, 3, 3])

In [56]:
w=z[:,0:3,0:3]
print(w.contiguous().shape)
w

torch.Size([2, 3, 3])


tensor([[[0.0761, 0.0382, 0.2798],
         [0.3035, 0.0857, 0.2666],
         [0.1222, 0.1765, 0.1431]],

        [[0.2141, 0.0117, 0.0949],
         [0.2689, 0.1247, 0.2399],
         [0.3284, 0.3797, 0.1583]]], grad_fn=<SliceBackward>)

In [57]:
w.contiguous().view(2,-1)

tensor([[0.0761, 0.0382, 0.2798, 0.3035, 0.0857, 0.2666, 0.1222, 0.1765, 0.1431],
        [0.2141, 0.0117, 0.0949, 0.2689, 0.1247, 0.2399, 0.3284, 0.3797, 0.1583]],
       grad_fn=<ViewBackward>)

In [58]:
w=torch.transpose(w.contiguous().view(2,-1),0,1)
w

tensor([[0.0761, 0.2141],
        [0.0382, 0.0117],
        [0.2798, 0.0949],
        [0.3035, 0.2689],
        [0.0857, 0.1247],
        [0.2666, 0.2399],
        [0.1222, 0.3284],
        [0.1765, 0.3797],
        [0.1431, 0.1583]], grad_fn=<TransposeBackward0>)

In [59]:
y=y[0,0:3,0:3].contiguous().view(1,-1)[0]#.shape

In [60]:
loss=nn.CrossEntropyLoss()

In [61]:
loss(w,y)

tensor(0.7134, grad_fn=<NllLossBackward>)

In [62]:
z=model(x.view(-1,1,128,128).float())

In [63]:
loss(torch.transpose(z[0].view(2,-1),0,1),dataset[0]['mask'].view(1,-1)[0])

tensor(0.6598, grad_fn=<NllLossBackward>)

In [96]:
Y=torch.ones((5,1,3,3),requires_grad=True)

In [141]:
Y

tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]], requires_grad=True)

In [142]:
Ypred=torch.rand((5,2,3,3),requires_grad=True)
Ypred

tensor([[[[0.6998, 0.3258, 0.3018],
          [0.7102, 0.5224, 0.6430],
          [0.4607, 0.6872, 0.4960]],

         [[0.7706, 0.2161, 0.6120],
          [0.6618, 0.7722, 0.5440],
          [0.0833, 0.2366, 0.8591]]],


        [[[0.9197, 0.1869, 0.6770],
          [0.1510, 0.4723, 0.8070],
          [0.9634, 0.4962, 0.9928]],

         [[0.0830, 0.7743, 0.2519],
          [0.3384, 0.8767, 0.9546],
          [0.6596, 0.7226, 0.1215]]],


        [[[0.8757, 0.0719, 0.6703],
          [0.1982, 0.1750, 0.6063],
          [0.1513, 0.4841, 0.5901]],

         [[0.3224, 0.8725, 0.3074],
          [0.2137, 0.3009, 0.6087],
          [0.6573, 0.5292, 0.4992]]],


        [[[0.8596, 0.5984, 0.8455],
          [0.0541, 0.9858, 0.3831],
          [0.7622, 0.6099, 0.5792]],

         [[0.3192, 0.2003, 0.7071],
          [0.1123, 0.8308, 0.0677],
          [0.4233, 0.2088, 0.5797]]],


        [[[0.0295, 0.9047, 0.4460],
          [0.9266, 0.4892, 0.2627],
          [0.4769, 0.5994, 0.8755]],

  

In [143]:
torch.exp(Ypred)[:,0]/torch.sum(torch.exp(Ypred),1)

tensor([[[0.4823, 0.5274, 0.4231],
         [0.5121, 0.4379, 0.5247],
         [0.5932, 0.6108, 0.4102]],

        [[0.6978, 0.3572, 0.6047],
         [0.4533, 0.4003, 0.4632],
         [0.5754, 0.4436, 0.7050]],

        [[0.6349, 0.3099, 0.5897],
         [0.4961, 0.4686, 0.4994],
         [0.3761, 0.4887, 0.5227]],

        [[0.6319, 0.5982, 0.5346],
         [0.4855, 0.5387, 0.5782],
         [0.5839, 0.5989, 0.4999]],

        [[0.3171, 0.6921, 0.3866],
         [0.7005, 0.6030, 0.4711],
         [0.4245, 0.4171, 0.6282]]], grad_fn=<DivBackward0>)

In [144]:
torch.sum(torch.exp(Ypred),1).shape

torch.Size([5, 3, 3])

In [145]:
torch.exp(Ypred).shape

torch.Size([5, 2, 3, 3])

In [146]:
def softmax(x):
    Out=torch.zeros(x.shape)
    Sum=torch.sum(torch.exp(Ypred),1)
    Out[:,0]=torch.exp(x)[:,0]/Sum
    Out[:,1]=torch.exp(x)[:,1]/Sum
    return Out

def logsoftmax(x):
    Out=torch.zeros(x.shape)
    Sum=torch.sum(torch.exp(Ypred),1)
    Out[:,0]=torch.exp(x)[:,0]/Sum
    Out[:,1]=torch.exp(x)[:,1]/Sum
    return torch.log(Out)

In [148]:
z=softmax(Ypred)
z.shape

torch.Size([5, 2, 3, 3])

In [200]:
z1=torch.softmax(Ypred,1)
z1.shape

torch.Size([5, 2, 3, 3])

In [182]:
torch.equal(z,z1)

False

In [183]:
z[0]

tensor([[[0.4823, 0.5274, 0.4231],
         [0.5121, 0.4379, 0.5247],
         [0.5932, 0.6108, 0.4102]],

        [[0.5177, 0.4726, 0.5769],
         [0.4879, 0.5621, 0.4753],
         [0.4068, 0.3892, 0.5898]]], grad_fn=<SelectBackward>)

In [184]:
z1[0]

tensor([[[0.4823, 0.5274, 0.4231],
         [0.5121, 0.4379, 0.5247],
         [0.5932, 0.6108, 0.4102]],

        [[0.5177, 0.4726, 0.5769],
         [0.4879, 0.5621, 0.4753],
         [0.4068, 0.3892, 0.5898]]], grad_fn=<SelectBackward>)

In [185]:
z1[0,0,0,0]

tensor(0.4823, grad_fn=<SelectBackward>)

In [186]:
torch.max(z-z1)

tensor(5.9605e-08, grad_fn=<MaxBackward1>)

In [187]:
torch.min(z-z1)

tensor(-5.9605e-08, grad_fn=<MinBackward1>)

In [202]:
lsm=nn.Softmax(1)

In [203]:
z2=lsm(Ypred)

In [205]:
torch.equal(z2,z1)

True

In [210]:
z=model(dataset[0]['image'].view(-1,1,128,128).float())

In [211]:
z.shape

torch.Size([1, 2, 128, 128])

In [218]:
y=dataset[0]['mask']
y.shape

torch.Size([1, 128, 128])

In [219]:
criterion=nn.CrossEntropyLoss()

In [222]:
criterion(z,y)

tensor(0.6598, grad_fn=<NllLoss2DBackward>)